### Azure Speech to Text streaming

Azure Speech to Text streaming allows streaming consumers of the AI speech service using web sockets.

#### Key Features

- **Integration with Azure Services**: Seamlessly integrate with other Azure services for enhanced functionality.
- **Customizable Assistants**: Create assistants tailored to specific needs with custom instructions and tools.
- **API Access**: Interact with assistants programmatically using the Azure OpenAI API.

#### Overview

1. **Setup and Configuration**:
    - Install necessary packages and import required modules.
    - Load environment variables and configure API keys.

By following these steps and utilizing the provided code snippets below, you can effectively manage and deploy Azure Speech Service to enhance your applications with AI-driven capabilities. For more detailed information, refer to the [Microsoft Learn documentation](http  s://learn.microsoft.com/en-us/azure/cognitive-services/openai/).

### Notebook Details
The following Python notebook will guide you through using the Azure Speech To Text streaming APIs.

#### Assumptions/Prerequisites
- You have an API key with access to the Azure Speech Streaming API in APIM.
- You have a user configuration file that specifies your access to AI services.



### Install the required packages

In [ ]:
%pip install python-dotenv azure-cognitiveservices-speech scipy utils

### Load the configuration settings

In [ ]:
from dotenv import dotenv_values, load_dotenv
import sys
sys.path.append('../common')
from helper import Helper
import os

env = os.getenv('GITHUB_ENV') or 'local'
env_name = os.getenv('ENV_NAME') or 'GO-AMER-USE-DV'

load_dotenv()

secrets = dotenv_values("secrets.env")
api_key = secrets.get("API_KEY")
local_env = secrets.get('LOCAL_URL') or secrets.get("APIM_NAME")
environment = secrets.get("APIM_NAME")
account_connection_string = secrets.get("AZURE_STORAGE_CONNECTION_STRING")
region_override = secrets.get("REGION_OVERRIDE")

deploymentsHelper = Helper(environment, "genai/azure/deployments", api_key, region_override)

deploymentsHelper.log('GITHUB_ENV: ' + env)
deploymentsHelper.log('ENV_NAME: ' + env_name)

### Speech recognition  

In [ ]:
"""
Speech recognition test using the Microsoft Cognitive Services Speech SDK
"""
from dotenv import dotenv_values, load_dotenv
import json
import string
import time
import threading
import wave
import utils
import sys
import io
import os

sys.path.append('../common')
from helper import Helper

try:
    import azure.cognitiveservices.speech as speechsdk
except ImportError:
    print("""
    Importing the Speech SDK for Python failed.
    Refer to
    https://docs.microsoft.com/azure/cognitive-services/speech-service/quickstart-python for
    installation instructions.
    """)
    sys.exit(1)

env_name = os.getenv('ENV_NAME') or 'GO-AMER-USE-DV'
secrets = dotenv_values("secrets.env")
speech_key = secrets.get("API_KEY")
environment = secrets.get("APIM_NAME")
region_override = secrets.get("REGION_OVERRIDE")

helper = Helper(environment, "genai/azure/deployments", api_key, region_override)

env_config = helper.read_json_config(env_name)
# Check if deployment_info is found
assert env_config is not None, f"Deployment information not found for environment: {env_name}"

speech_endpoint = "wss://" +  env_config["host_name"] + "/genai/azure/speech/streaming?x-kpmg-charge-code=1223232"

# Specify the path to an audio file containing speech (mono WAV / PCM with a sampling rate of 16
# kHz).
weatherfilename = "./data/whatstheweatherlike.wav"

def speech_recognize_once_from_file_with_detailed_recognition_results():
    """performs one-shot speech recognition with input from an audio file, showing detailed recognition results
    including word-level timing """
    # <SpeechRecognitionFromFileWithDetailedRecognitionResults>
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, endpoint=speech_endpoint)

    # Ask for detailed recognition result
    speech_config.output_format = speechsdk.OutputFormat.Detailed

    # If you also want word-level timing in the detailed recognition results, set the following.
    # Note that if you set the following, you can omit the previous line
    #   "speech_config.output_format = speechsdk.OutputFormat.Detailed",
    # since word-level timing implies detailed recognition results.
    speech_config.request_word_level_timestamps()

    audio_config = speechsdk.audio.AudioConfig(filename=weatherfilename)

    # Creates a speech recognizer using a file as audio input, also specify the speech language
    speech_recognizer = speechsdk.SpeechRecognizer(
        speech_config=speech_config, language="en-US", audio_config=audio_config)

    # Starts speech recognition, and returns after a single utterance is recognized. The end of a
    # single utterance is determined by listening for silence at the end or until a maximum of about 30
    # seconds of audio is processed. It returns the recognition text as result.
    # Note: Since recognize_once() returns only a single utterance, it is suitable only for single
    # shot recognition like command or query.
    # For long-running multi-utterance recognition, use start_continuous_recognition() instead.
    result = speech_recognizer.recognize_once()

    # Check the result
    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        helper.log('All good!!!')
        assert True
    elif result.reason == speechsdk.ResultReason.NoMatch:
        helper.log("No speech could be recognized: {}".format(result.no_match_details))
    elif result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = result.cancellation_details
        helper.log("Speech Recognition canceled: {}".format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            helper.log("Error details: {}".format(cancellation_details.error_details))
            assert False
    # </SpeechRecognitionFromFileWithDetailedRecognitionResults>

try:
    speech_recognize_once_from_file_with_detailed_recognition_results();
except Exception as e:
    helper.log('Error running sample: {}'.format(e))
    assert False
